In [1]:
from typing import List
from functools import wraps

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
import datetime as dt

jtplot.style(figsize=(15, 9))

In [2]:
# tickers = ['GLD', 'GDX', 'AAPL', 'SPY']
# # tickers = ['MSFT', 'NVDA', 'AMD', 'META']
# d_start = dt.datetime(2010, 1, 1)
# d_final = dt.datetime(2020, 12, 31)

# df = yf.download(tickers, start=d_start, end=d_final, period='1d', auto_adjust=True)
# df.to_csv(f"../../../data/bt/{'_'.join(tickers)}__1D.csv")

In [2]:
import os

DATA_PATH = '../../data/bt/'

def read_csv_bt(csv:str)->pd.DataFrame:
    file_path = f"{DATA_PATH}{csv}"
    tickers, _ = csv.split('__')
    if len(tickers)>1:
        return pd.read_csv(file_path, parse_dates=True, header=[0, 1], index_col=0)['Close']
    else:
        return pd.read_csv(file_path, parse_dates=True, index_col=0)['Close']

def get_backtest_data()->List[list]:
    bt_files = os.listdir(DATA_PATH)
    csv_files = []
    for csv in bt_files:
        try:
            tickers, period = csv.split('__')
            df = read_csv_bt(csv) # send to some dict with data???
            csv_files.append({
                'csv': csv,
                'tickers': list(map(str.upper, tickers.split('_'))),
                'period': period.replace('.csv', ''),
                'start': df.iloc[0].name.strftime('%d/%m/%Y'),
                'end': df.iloc[-1].name.strftime('%d/%m/%Y')
            })
        except:
            continue
    return csv_files


## Strategies for testing

In [3]:
class Strategy:
    def __init__(self):
        pass
    
    def set_data(self, ticker:str, d:pd.DataFrame):
        self.d = d.copy()
        self.d['returns'] = np.log(d[ticker] / d[ticker].shift(1))
        self.ticker = ticker
        self.strategy = []
        self.drawdown = []


In [4]:
class SMA(Strategy):
    def __init__(self, sma1:List[int]=[42], sma2:List[int]=[252]):
        self.sma1 = sma1
        self.sma2 = sma2
    
    def run_strategy(self):
        for s1, s2 in zip(self.sma1, self.sma2):
            self.d[f'SMA1_{s1}'] = self.d[self.ticker].rolling(s1).mean()
            self.d[f'SMA2_{s2}'] = self.d[self.ticker].rolling(s2).mean()
            self.d.dropna(inplace=True)

            self.d[f'position_{s1}_{s2}'] = np.where(self.d[f'SMA1_{s1}'] > self.d[f'SMA2_{s2}'], 1, -1)
            self.d[f'strategy_{s1}_{s2}'] = self.d[f'position_{s1}_{s2}'].shift(1) * self.d['returns']
            self.strategy.append(f'strategy_{s1}_{s2}')

            self.d[f'cumret_{s1}_{s2}'] = self.d[f'strategy_{s1}_{s2}'].cumsum().apply(np.exp)
            self.d[f'cummax_{s1}_{s2}'] = self.d[f'cumret_{s1}_{s2}'].cummax()
            self.d[f'drawdown_{s1}_{s2}'] = self.d[f'cummax_{s1}_{s2}'] - self.d[f'cumret_{s1}_{s2}']
            self.drawdown.append(f'drawdown_{s1}_{s2}')

        strategy_result = self.d[self.strategy+['returns']].sum().apply(np.exp)
        drawdown_result = self.d[self.drawdown].max()
        result = pd.concat([strategy_result, drawdown_result]).to_frame()
        result.rename(columns={0: self.ticker}, inplace=True)

        return result, self.d.copy()
    

In [10]:
class MOM(Strategy):
    def __init__(self, moms:List[int]=[1]):
        self.moms = moms
    
    def run_strategy(self):
        for mom in self.moms:
            self.d[f'MOM_{mom}'] = self.d[self.ticker].rolling(mom).mean()
            self.d.dropna(inplace=True)

            self.d[f'position_{mom}'] = np.sign(self.d['returns'].rolling(mom).mean())
            self.d[f'strategy_{mom}'] = self.d[f'position_{mom}'].shift(1) * self.d['returns']
            self.strategy.append(f'strategy_{mom}')

            self.d[f'cumret_{mom}'] = self.d[f'strategy_{mom}'].cumsum().apply(np.exp)
            self.d[f'cummax_{mom}'] = self.d[f'cumret_{mom}'].cummax()
            self.d[f'drawdown_{mom}'] = self.d[f'cummax_{mom}'] - self.d[f'cumret_{mom}']
            self.drawdown.append(f'drawdown_{mom}')

        strategy_result = self.d[self.strategy+['returns']].sum().apply(np.exp)
        drawdown_result = self.d[self.drawdown].max()
        result = pd.concat([strategy_result, drawdown_result]).to_frame()
        result.rename(columns={0: self.ticker}, inplace=True)
    
        return result, self.d.copy()


In [11]:
def start_backtesting(strategy_cls, *args)->list:
    bt_results_dfs = []
    bt_data_files = get_backtest_data()
    for bt in bt_data_files:
        csv, tickers, period, _, _ = bt.values()
        data = read_csv_bt(csv)

        result_df = pd.DataFrame()
        single_ticker_result = []
        STRATEGY = get_strategy(strategy_cls, *args)
        for ticker in tickers:
            STRATEGY.set_data(ticker, pd.DataFrame(data[ticker]))
            result, full_data = STRATEGY.run_strategy()
            
            single_ticker_result.append(result)

        single_ticker_result = pd.concat(single_ticker_result, axis=1)
        single_ticker_result.index.name = period
        bt_results_dfs.append(single_ticker_result)
    return bt_results_dfs
        
def get_strategy(strategy, *args):
    strategies = dict(sma=SMA(*args), mom=MOM(*args))
    return strategies[strategy]


In [42]:
SMA1 = [42, 24, 18, 6]
SMA2 = [252, 180, 64, 22]
start_backtesting('sma', SMA1, SMA2)

[                      GLD       GDX       AAPL       SPY
 1D                                                      
 strategy_42_252  1.308799  1.065529   6.424632  1.466223
 strategy_24_180  1.162980  1.102915   6.462446  1.668211
 strategy_18_64   1.611339  0.746799   3.299714  1.804156
 strategy_6_22    1.462546  2.269095   5.655584  0.803092
 returns          1.107165  0.732320  10.219678  3.437358
 drawdown_42_252  0.447434  1.041484   4.107096  0.623526
 drawdown_24_180  0.402130  0.787659   1.729420  1.023388
 drawdown_18_64   0.308936  1.408235   1.144106  0.520338
 drawdown_6_22    0.322727  2.556176   1.345458  0.557455,
                       GLD       GDX      AAPL       SPY
 1H                                                     
 strategy_42_252  1.210578  0.547866  1.141445  1.205247
 strategy_24_180  1.001059  0.695432  1.112150  1.183577
 strategy_18_64   1.258914  1.151021  1.049129  1.315175
 strategy_6_22    0.788741  0.802870  0.819486  1.026587
 returns          1

In [12]:
start_backtesting('mom')

[                 GLD       GDX       AAPL       SPY
 1D                                                 
 strategy_1  0.597487  0.028094   2.176327  0.713846
 returns     1.618397  0.824147  20.280812  4.083878
 drawdown_1  0.462040  1.000328   4.992419  0.549679,
                  GLD       GDX      AAPL       SPY
 1H                                                
 strategy_1  0.871211  1.321527  1.189769  1.017373
 returns     1.538725  1.591653  1.594131  1.570848
 drawdown_1  0.252276  0.352044  0.295401  0.187019,
                 MSFT       NVDA       AMD      META
 1D                                                 
 strategy_1  0.125920   0.086758  0.416052  0.293725
 returns     9.196292  30.931284  9.514433  7.111431
 drawdown_1  1.140283   1.000582  9.252131  1.450128,
                 MSFT       NVDA       AMD      META
 1H                                                 
 strategy_1  0.126422   0.086756  0.416052  0.293724
 returns     9.196294  30.931282  9.514433  7.11